# Import needs

In [19]:
import os
import json
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

# Start transformations

The reason we need transformations is that our data can't be used directly by the model, we need to transform it to the right format. More specifically, `the VOC format`

In [3]:
train = pd.read_csv('data/train.csv')
train.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]


In [11]:
for i, row in train.iterrows():
    print(row)
    print(row.annotations)
    break

video_id              0
sequence          40258
video_frame           0
sequence_frame        0
image_id            0-0
annotations          []
Name: 0, dtype: object
[]


In [30]:
i = 64
annotations = train.iloc[i].annotations
print(train.iloc[i].video_frame)
annotations

64


"[{'x': 458, 'y': 16, 'width': 57, 'height': 57}, {'x': 522, 'y': 572, 'width': 141, 'height': 126}]"

In [18]:
#transform annotations to list
annotations = annotations.replace("\'", "\"")
annotations_json = json.loads(annotations)
annotations_json

[{'x': 539, 'y': 205, 'width': 105, 'height': 98},
 {'x': 630, 'y': 351, 'width': 95, 'height': 56}]

# XML tests

In [64]:
# hello = ET.Element("hello")
# ET.SubElement(hello, "world")
# ET.ElementTree(hello).write("hello.xml")

# VOC format

In [43]:
def create_voc_format(df, out_dir, source_images_dir="data/train_images"):

    def get_json_annotation(annotation):
        return json.loads(annotation.replace("\'", "\""))
    
    def _create_object_subtree(annotation, obj):
        """ Create the sub-tree related to a given object and update the root """
        # Object Sub-Element
        # Check that bbox is not NaN
        _object = ET.SubElement(annotation, "object")
        ET.SubElement(_object, "name").text = "target"
        # ET.SubElement(_object, "radiologist").text = obj[2]
        ET.SubElement(_object, "pose").text = "Unspecified"
        ET.SubElement(_object, "truncated").text = "0"
        ET.SubElement(_object, "difficult").text = "0"

        _bndbox = ET.SubElement(_object, "bndbox")
        ET.SubElement(_bndbox, "xmin").text = str(int(obj["x"]))
        ET.SubElement(_bndbox, "ymin").text = str(int(obj["y"]))
        ET.SubElement(_bndbox, "xmax").text = str(int(obj["x"] + obj["width"]))
        ET.SubElement(_bndbox, "ymax").text = str(int(obj["y"] + obj["height"]))
        return annotation
    def build_prefix(row):
        return str(row.video_id) +"_" +str(row.video_frame)
    #check if the directory exists, if not create it
    annotation_dir = out_dir+ "/Annotations"
    if not os.path.exists(annotation_dir):
        os.makedirs(annotation_dir)
    image_dir = out_dir+ "/JPEGImages"
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
    split_dir = out_dir+ "/ImageSets/Main"
    if not os.path.exists(split_dir):
        os.makedirs(split_dir)
    # add train.txt test.txt val.txt trainval.txt 
    for file_name in ["train.txt", "test.txt", "val.txt", "trainval.txt"]:
        with open(os.path.join(split_dir, file_name), "w") as f:
            f.write("")
        
        
    #copy images from data to image_directory and add them to split
    for i, row in df.iterrows():
        src_path = os.path.join(source_images_dir, 
                                "video_" + str(row.video_id),
                                str(row.video_frame) + ".jpg")
        save_path = os.path.join(image_dir, build_prefix(row) + ".jpg")
        os.system("cp " + src_path + " " + save_path)
        for file_name in ["train.txt", "trainval.txt"] :
            with open(os.path.join(split_dir, file_name), "a") as f:
                f.write(build_prefix(row) + "\n")
        
        
    
        
        
    for index, row in df.iterrows():
        prefix = build_prefix(row)
        # Initalize and create the objects array
        objects = get_json_annotation(row.annotations)
        annotation = ET.Element('annotation')
        # img_w, img_h, img_d = _get_image_size(os.path.join(train_dir, image_id+".dicom"))
        img_w, img_h, img_d = "1280", "720", "1"
        # img_w, img_h, w_ratio, h_ratio = resize(img_w, img_h)
        
        # ##### Beginning of XML #####
        
        # File Sub-Elements
        ET.SubElement(annotation, "folder").text="train"
        ET.SubElement(annotation, "filename").text= prefix +".jpg"
        # ET.SubElement(annotation, "path").text=os.path.join(train_dir, prefix+".dicom")
        ET.SubElement(annotation, "path").text= prefix+".jpg"

        # SRC Sub-Element
        _src = ET.SubElement(annotation, "src")
        ET.SubElement(_src, "database").text="train"
        
        # Size Sub-Element
        _size = ET.SubElement(annotation, "size")
        ET.SubElement(_size, "width").text=str(img_w)
        ET.SubElement(_size, "height").text=str(img_h)
        ET.SubElement(_size, "depth").text=str(img_d)
        
        # Segmented Sub-Element
        ET.SubElement(annotation, "segmented").text="0"
        
        # Loop over every object and create the annotation for that bounding box
        for obj in objects:
            annotation = _create_object_subtree(annotation, obj)
                    
        # Save to output directory
        ET.ElementTree(annotation).write(os.path.join(annotation_dir, prefix+'.xml'))
    
# DEMO_IMG_ID = "9a5094b2563a1ef3ff50dc5c7ff71345"
# DEMO_PATH = os.path.join(TRAIN_DIR, DEMO_IMG_ID+".dicom")

create_voc_format(train, out_dir="./YOLOX/datasets/VOCdevkit/VOC2012")

# print("Look into the tmp folder to see that the xml was created...")
# for file in [f for f in os.listdir("/tmp") if f.endswith(".xml")]: print("\t– /tmp/"+file)

In [42]:
# %rm -dfr ./YOLOX/datasets/VOCdevkit/VOC2012